In [ ]:
<img src="files/image.png">

In [1]:
def delay_coordinate_embedding(data_file,tau,m):
    time_series_data = np.array([])
    for line in data_file:
        time_series_data = np.append(time_series_data,float(line))
    projection = np.array([])
    for i in np.arange((m-1)*tau,time_series_data.shape[0],1):
        delay_coordinate = np.array([])
        for j in np.arange(0,m*tau,tau):
            delay_coordinate = np.append(delay_coordinate,time_series_data[i-j])
        if(i == (m-1)*tau):
            projection = delay_coordinate
        projection = np.column_stack((projection,delay_coordinate))
    return projection

* Used TISEAN to produce a surrogate timeseries of polarity scores.
* False nearest neighbors gave best dimension as 5
* mutual information gave best tau as 100

In [ ]:
import numpy as np 
df = open("../twitractors/surrogate_polarity_germex.csv",'r')
embedding = delay_coordinate_embedding(df,5,100)
plt.plot(embedding[0,:],embedding[1,:])
plt.show()